In [1]:
! pip install datasets transformers nltk

  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.5 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (31 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
Using cached aiohttp-3.9.5-cp312-cp312-macosx_11_0_arm64.whl (392 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 15.6 MB/s eta 0:00:00m eta 0:00:010:01:01
Using cached click-8.1.7-py3-none-any.whl (97 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import (AutoModelForSeq2SeqLM,
                          AutoTokenizer,
                          T5Tokenizer)
import torch
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk
from tqdm import tqdm

#import subprocess
import sys
import os
import argparse
from IPython.core import error
import random
import numpy as np
import nltk
import json
import csv

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [15]:
output_dir = './'
# Here you need to link this path in your Google Drive to the place preserving your model weights, e.g., checkpoint-62500
# You can download it on the GitHub page

model_checkpoint = "t5-base"
prefix = "Transform the following sentence into Signal Temporal logic: "

max_input_length = 1024
max_target_length = 128
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length=max_input_length)

match true:
    case torch.cuda.is_available():
        device = torch.device("cuda:0")
    case torch.backends.mps.is_available():
        device = torch.device("mps")
    case _:
        device = torch.device("cpu")

tl_model = AutoModelForSeq2SeqLM.from_pretrained(output_dir + "checkpoint-62500").to(device)

In [11]:
import time

time_start = time.time()
inputs = [prefix + 'At some point (prop_1), and at some point (prop_2), and always do not (prop_4).']
inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
output = tl_model.generate(**inputs, num_beams=8, do_sample=True, max_length=max_target_length)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print(decoded_output)
time_end = time.time()
print('Translation time: ', time_end - time_start)

( ( prop_1 and prop_2 ) and globally ( negation prop_4 ) )
Translation time:  14.548202991485596


In [14]:
# Here are the example test sentences
test_sentence = [
    'Stay at (prop_1) for 5 units in the future and stay at (prop_2) for 5 units in the future, and ensure that never (prop_3).',
    'First (prop_1), and then (prop_2), and ensure that never (prop_3).',
    'Start by (prop_1). Then, (prop_2). Lastly, (prop_3).',
    'Guarantee that you (prop_1) and (prop_2)',  # Input the natural sentence
    '( prop_1 ) and whenever ( prop_2 )',
    'Sooner or later (prop_1)',
    'Repeatedly (prop_1)',
    'At some point, (prop_1).',
    'Do prop_1 but not do prop_2',
    'Do prop_1, do prop_2, do prop_3']  # Input the natural sentence

for sentence in test_sentence:
    inputs = [prefix + sentence]
    inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
    output = tl_model.generate(**inputs, num_beams=8, do_sample=True, max_length=max_target_length)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    print('Input sentence: ', sentence)
    print('Translated STL: ', decoded_output)
    print('\n')

Input sentence:  Stay at (prop_1) for 5 units in the future and stay at (prop_2) for 5 units in the future, and ensure that never (prop_3).
Translated STL:  ( ( globally [0,5] prop_1 and globally [0,5] prop_2 ) and globally [0,1] negation prop_3 )


Input sentence:  First (prop_1), and then (prop_2), and ensure that never (prop_3).
Translated STL:  ( ( prop_1 until prop_2 ) and globally ( negation prop_3 ) )


Input sentence:  Start by (prop_1). Then, (prop_2). Lastly, (prop_3).
Translated STL:  ( ( prop_1 until prop_2 ) until prop_3 )


Input sentence:  Guarantee that you (prop_1) and (prop_2)
Translated STL:  finally ( prop_1 and prop_2 )


Input sentence:  ( prop_1 ) and whenever ( prop_2 )
Translated STL:  ( prop_1 and finally prop_2 )


Input sentence:  Sooner or later (prop_1)
Translated STL:  finally prop_1


Input sentence:  Repeatedly (prop_1)
Translated STL:  globally ( finally prop_1 )


Input sentence:  At some point, (prop_1).
Translated STL:  finally prop_1


Input senten

In [16]:
sentence = "Guarantee that you (prop_1) and (prop_2)"

inputs = [prefix + sentence]
inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
output = tl_model.generate(**inputs, num_beams=8, do_sample=True, max_length=max_target_length)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print('Input sentence: ', sentence)
print('Translated STL: ', decoded_output)
print('\n')

Input sentence:  Guarantee that you (prop_1: keep the heat above 20 degrees Celsius) and (prop_2: keep the heat below 23 degrees Celsius)
Translated STL:  ( globally [20,infinite] prop_1 and ( negation prop_2 ) equal 23 degrees Celsius )


